In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
from sklearn.model_selection import train_test_split
import shutil
import pathlib

In [ ]:
# Extract the dataset
zip_path = "brain_tumor_dataset.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

In [3]:
# Define dataset directory
extract_path="brain_tumor_dataset"
dataset_dir = pathlib.Path(extract_path)



In [4]:
# Create train-test split directories
train_dir = os.path.join(extract_path, "train")
val_dir = os.path.join(extract_path, "val")

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

    

In [5]:
# Get all image paths and labels 
image_paths = list(dataset_dir.glob("**/*.jpg")) + list(dataset_dir.glob("**/*.png"))
labels = [os.path.basename(os.path.dirname(p)) for p in image_paths]


In [6]:
# Split dataset
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, stratify=labels, random_state=42)



In [7]:
# Function to move images to respective directories
def move_images(image_paths, labels, target_dir):
    for path, label in zip(image_paths, labels):
        label_dir = os.path.join(target_dir, label)
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)
        shutil.copy(str(path), os.path.join(label_dir, os.path.basename(path)))

move_images(train_paths, train_labels, train_dir)
move_images(val_paths, val_labels, val_dir)



In [8]:
# Image preprocessing
img_size = (150, 150)
batch_size = 32

datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)



Found 3195 images belonging to 2 classes.
Found 799 images belonging to 2 classes.


In [9]:
# Build CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])+


C:\Users\sreej\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)



C:\Users\sreej\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.7109 - loss: 0.7247 - val_accuracy: 0.9086 - val_loss: 0.2103
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 58s 574ms/step - accuracy: 0.9091 - loss: 0.2159 - val_accuracy: 0.9224 - val_loss: 0.1783
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 559ms/step - accuracy: 0.9499 - loss: 0.1289 - val_accuracy: 0.9462 - val_loss: 0.1502
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 57s 573ms/step - accuracy: 0.9547 - loss: 0.1041 - val_accuracy: 0.9574 - val_loss: 0.1225
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 597ms/step - accuracy: 0.9686 - loss: 0.0755 - val_accuracy: 0.9587 - val_loss: 0.1226
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 606ms/step - accuracy: 0.9918 - loss: 0.0314 - val_accuracy: 0.9712 - val_loss: 0.0944
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 58s 583ms/step - accuracy: 0.9945 - loss: 0.0160 - val_accuracy: 0.9687 - val_loss: 0.1463
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 667ms/step - accuracy: 0.9961 - loss: 0.0

In [11]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 215ms/step - accuracy: 0.9740 - loss: 0.1914
Validation Accuracy: 97.87%


In [12]:
model.save("brain_tumor_model.h5")

In [17]:
# user defined function for prediting a user input image
from tensorflow.keras.preprocessing.image import  load_img, img_to_array
import numpy as np

def predict_image(model, image_path):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions
    prediction = model.predict(img_array)[0][0]  # Get prediction value
    return "Tumor Detected" if prediction > 0.5 else "No Tumor Detected"

# Example usage
result = predict_image(model, "Tr-me_1301.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Tumor Detected
